#**Assignment 4**

##**Imports**

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

from scipy.sparse import coo_matrix
from scipy.sparse.linalg import spsolve

import numba
from numba import njit, prange, cuda

##**CPU explicit method - Forward Euler method**

In [ ]:
class explicit_heat_equation():
  def __init__(self, N, iterations, length, alpha):
    self.N = N
    self.iterations = iterations
    self.u = np.zeros((iterations, N, N))
    self.length = length
    self.alpha = alpha

    self.initial = 0.0
    self.right = 0.0
    self.left = 0.0
    self.top = 0.0
    self.bottom = 0.0
    
    self.dh = length / (N-1)
    self.dt = (self.dh ** 2)/(4 * alpha)
    self.gamma = (alpha * self.dt) / (self.dh ** 2)

    if self.dt < self.dh**2/4:
      raise ValueError('Stability condition is not met')
  
  def set_boundary_conditions(self, initial, top, left, right, bottom):

    self.initial = initial
    self.right = right
    self.left = left
    self.top = top
    self.bottom = bottom

    # Set the initial condition
    self.u.fill(initial)

    # Set the boundary conditions
    self.u[:, -1:, :] = u_bottom
    self.u[:, :, :1] = u_left
    self.u[:, :1, 1:] = u_top
    self.u[:, :,-1:] = u_right

  def calculate_u(self):
    for k in range(self.iterations - 1):
      for i in range(1, self.N - 1):
        for j in range(1, self.N - 1):
          self.u[k + 1, i, j] = (1 - 4 * self.gamma) * self.u[k][i][j] + self.gamma * (self.u[k][i+1][j] + self.u[k][i-1][j] + self.u[k][i][j+1] + self.u[k][i][j-1])
          
  def calculate_middle_point(self, value):
    t = 0
    for j in range(self.iterations):
      if self.N % 2 == 0:
        index = self.N // 2
        average_mid_point = self.u[j][index][index] + self.u[j][index-1][index-1] + self.u[j][index-1][index] + self.u[j][index][index-1] / 4
        if average_mid_point >=1:
          t = j
          break

      elif self.N % 2 != 0:
        index = self.N // 2
        if self.u[j][index][index] >= value:
          t = j
          break

    if t == 0:
      raise ValueError('Middle value does not converge to value')

    return t

  def plot_heat_map(self, iteration):
    plt.imshow(self.u[iteration], cmap='hot', interpolation='nearest')
    plt.show()
      

In [ ]:
def plot_convergence(N_list):
  iterations = np.zeros(len(N_list))
  for i in range(len(N_list)):
    N = N_list[i]
    heat_problem = explicit_heat_equation(N, iter, length, alpha)
    heat_problem.set_boundary_conditions(u_initial, u_top, u_left, u_right, u_bottom)
    heat_problem.calculate_u()
    t = heat_problem.calculate_middle_point(value)
    iterations[i] = t * heat_problem.dt
    
  plt.plot(N_list, iterations)
  plt.show()

In [ ]:
N = 51
iter = 2000
length = 2
alpha = 1
value = 1

#Initial temperature
u_initial = 0

# Boundary conditions (fixed temperature)
u_top = 0.0
u_left = 5.0
u_bottom = 0.0
u_right = 0.0

In [ ]:
heat_problem = explicit_heat_equation(N, iter, length, alpha)
heat_problem.set_boundary_conditions(u_initial, u_top, u_left, u_right, u_bottom)
heat_problem.calculate_u()
t = heat_problem.calculate_middle_point(value)

In [ ]:
t

1059

In [ ]:
n = [i for i in range(51, 102, 10)]
plot_convergence(n)

ValueError: ignored

##**CPU implicit method - Backward Euler method**

In [ ]:
class implicit_heat_equation():
  def __init__(self, N, iterations, length, alpha):
    self.N = N
    self.iterations = iterations
    self.u = np.zeros((iterations, N**2))
    self.length = length
    self.alpha = alpha

    self.initial = 0.0
    self.right = 0.0
    self.left = 0.0
    self.top = 0.0
    self.bottom = 0.0
    
    self.A = np.zeros((N**2, N**2))
    self.dh = length / (N-1)
    self.dt = (self.dh ** 2)/(4 * alpha)
    self.gamma = (alpha * self.dt) / (self.dh ** 2)

    if self.dt < self.dh**2 / 4:
      raise ValueError('Stability condition is not met')
  
  def set_boundary_conditions(self, initial, top, left, right, bottom):

    self.right = right
    self.left = left
    self.top = top
    self.bottom = bottom

    # Set the boundary conditions
    self.u[:,::self.N]= left
    self.u[:, self.N-1::self.N] = right
    self.u[:, -self.N:] = bottom
    self.u[:,:self.N] = top

  def calc_A_matrix(self):

    for i in range(self.N**2):
      #update main diagonal
      self.A[i,i] = 1 + 4 * self.gamma
      #update
      if i < self.N**2 - 1:
        self.A[i,i+1] = - self.gamma
      #update
      if i >= self.N:
        self.A[i, i-self.N] = - self.gamma
      #update
      if i <  self.N**2 - self.N:
        self.A[i, i+self.N] = - self.gamma
      #update lower diagonal
      if i >= 1:
        self.A[i,i-1] = - self.gamma

  def iterate_u(self):

    for k in range(self.iterations-1):
      self.u[k+1] = spsolve(self.A, self.u[k])

      # Set the boundary conditions
      self.u[k+1][::self.N]= self.left
      self.u[k+1][self.N-1::self.N] = self.right
      self.u[k+1][-self.N:] = self.bottom
      self.u[k+1][:self.N] = self.top

      self.u[k] = self.u[k+1]

  def calculate_middle_point(self, value):
    t = 0
    for j in range(self.iterations):
      index = int(self.N **2 / 2)
      if self.u[j][index] >= value:
        t = j
        break

    if t == 0:
      raise ValueError('Middle value does not converge to value')

    return t

  def plot_heat_map(self, iteration):
    plt.imshow(self.u[iteration].reshape((self.N, self.N)), cmap='hot', interpolation='nearest')
    plt.show()

In [ ]:
N = 51
iter = 1100
length = 2
alpha = 1
value = 1

#Initial temperature
u_initial = 0

# Boundary conditions (fixed temperature)
u_top = 0.0
u_left = 5.0
u_bottom = 0.0
u_right = 0.0

In [ ]:
heat_problem2 = implicit_heat_equation(N, iter, length, alpha)
heat_problem2.set_boundary_conditions(u_initial, u_top, u_left, u_right, u_bottom)
heat_problem2.calc_A_matrix()
heat_problem2.u.shape


(1100, 2601)

In [ ]:
heat_problem2.iterate_u()

/usr/local/lib/python3.7/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:138: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)


In [ ]:
heat_problem2.u.shape

(1100, 2601)

In [ ]:
i = heat_problem2.calculate_middle_point(value)
i * heat_problem2.dt

0.4348

## **GPU explicit method**

In [ ]:
@cuda.jit()
def kernel(u, gamma, N, iterations):
  i = cuda.grid(1)
  print(i)
  if i < N-1:
    # load the data into the shared memory
    local_u = cuda.const.array_like(u)
    local_gamma = float(gamma)

    cuda.syncthreads()
    for k in range(iterations - 1):
      for j in range(1, N-1):
        local_u[k + 1, i, j] = (1 - 4 * local_gamma) * local_u[k][i][j] + local_gamma * (local_u[k][i+1][j] + local_u[k][i-1][j] + local_u[k][i][j+1] + local_u[k][i][j-1])
        #local_u[:, -1:, :] = 0.0
        #local_u[:, :, :1] = 0.0
        #local_u[:, :1, 1:] = 5.0
        #local_u[:, :,-1:] = 0.0

      cuda.syncthreads()

In [ ]:
N = 5
iter = 10
length = 2
alpha = 1
value = 1

#Initial temperature
u_initial = 0

# Boundary conditions (fixed temperature)
u_top = 0.0
u_left = 5.0
u_bottom = 0.0
u_right = 0.0

heat_problem = explicit_heat_equation(N, iter, length, alpha)
heat_problem.set_boundary_conditions(u_initial, u_top, u_left, u_right, u_bottom)
u = heat_problem.u
gamma=heat_problem.gamma

In [ ]:
TPB = 16
BPG = (N+TPB-1)//TPB
#send 1D data to device
dev_u = cuda.to_device(u)

In [ ]:
kernel[(BPG,1), (TPB,1)](dev_u, gamma, N, iter)

In [ ]:
ary = np.empty(shape = u.shape,)
dev_u.copy_to_host(ary)  #call result to the device
dev_u = ary

In [ ]:
dev_u

array([[[5.        , 0.        , 0.        , 0.        , 0.        ],
        [5.        , 0.        , 0.        , 0.        , 0.        ],
        [5.        , 0.        , 0.        , 0.        , 0.        ],
        [5.        , 0.        , 0.        , 0.        , 0.        ],
        [5.        , 0.        , 0.        , 0.        , 0.        ]],

       [[5.        , 1.25      , 0.        , 0.        , 0.        ],
        [5.        , 1.25      , 0.        , 0.        , 0.        ],
        [5.        , 1.25      , 0.        , 0.        , 0.        ],
        [5.        , 1.25      , 0.        , 0.        , 0.        ],
        [5.        , 0.        , 0.        , 0.        , 0.        ]],

       [[5.        , 1.5625    , 0.3125    , 0.        , 0.        ],
        [5.        , 1.875     , 0.3125    , 0.        , 0.        ],
        [5.        , 1.875     , 0.3125    , 0.        , 0.        ],
        [5.        , 1.5625    , 0.3125    , 0.        , 0.        ],
        [5.     